In [7]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('../results.csv')

print(f"df shape {df.shape}")

# Print the DataFrame
df.head()

df shape (263, 6)


,id,player_1_model,player_1_temperature,player_2_model,player_2_temperature,player_1_won
0,20240323212519,mistral:mistral-large-latest,0.0,openai:gpt-4-0125-preview,0.0,False
1,20240323220117,grok:mixtral-8x7b-32768,0.0,openai:gpt-4-0125-preview,0.0,False
2,20240323220251,grok:mixtral-8x7b-32768,0.0,openai:gpt-4-0125-preview,0.0,True
3,20240323221351,grok:mixtral-8x7b-32768,0.0,openai:gpt-4-0125-preview,0.0,False
4,20240323221638,mistral:mistral-small-latest,0.0,mistral:mistral-small-latest,0.0,True


In [17]:
# Strip leading and trailing spaces from column names
df.columns = df.columns.str.strip()

# Do the same on the text values
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Now, your existing code should work
import matplotlib.pyplot as plt

# Check if 'winner' column exists in the DataFrame
if 'winner' in df.columns:
    # Ensure 'winner' column is of boolean type
    df['winner'] = df['winner'].astype(bool)

    # Filter the DataFrame based on the condition
    winner_counts = df[df['winner'] == True]['player_1'].value_counts()

    # Plot the count of occurrences for each player
    winner_counts.plot(kind='bar')

    # Set the plot title and labels
    plt.title('Number of Wins for Each Player')
    plt.xlabel('Player')
    plt.ylabel('Count')

    # Show the plot
    plt.show()
else:
    print("Column 'winner' does not exist in the DataFrame.")

Column 'winner' does not exist in the DataFrame.


In [16]:
# Get the different contending models
players = df['player_1_model'].unique()
print(f"Players: {players}")

# Compute the ELO 


Players: ['mistral:mistral-large-latest' 'grok:mixtral-8x7b-32768'
 'mistral:mistral-small-latest']


In [ ]:
# Initialize player ratings
player_ratings = {player_id: 1500 for player_id in pd.concat([df['player_1_model'], df['player_1_model']]).unique()}

def elo_expected_score(rating_a, rating_b):
    return 1 / (1 + 10**((rating_b - rating_a) / 400))

def elo_update(winner_rating, loser_rating, k=32):
    expected_score_winner = elo_expected_score(winner_rating, loser_rating)
    expected_score_loser = 1 - expected_score_winner
    new_winner_rating = winner_rating + k * (1 - expected_score_winner)
    new_loser_rating = loser_rating + k * (0 - expected_score_loser)
    return new_winner_rating, new_loser_rating

# Iterate through matches to update ELO ratings
for index, row in df.iterrows():
    player1, player2, player1_won = row['player_1_model'], row['player_1_model'], row['player_1_won']
    
    if player1_won:
        winner, loser = player1, player2
    else:
        winner, loser = player2, player1
    
    new_winner_rating, new_loser_rating = elo_update(player_ratings[winner], player_ratings[loser])
    
    player_ratings[winner] = new_winner_rating
    player_ratings[loser] = new_loser_rating

# Print updated ratings
print(player_ratings)